In [1]:
! nvidia-smi

Wed Jul 16 07:27:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.04              Driver Version: 576.52         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090 Ti     On  |   00000000:01:00.0  On |                  Off |
|  0%   43C    P8             29W /  450W |   14059MiB /  24564MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch

print(f"CUDA 사용여부 : {torch.cuda.is_available()}") 
print("사용 가능한 GPU 수:", torch.cuda.device_count())

if torch.cuda.is_available():
    print("현재 사용 중인 GPU:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA를 사용할 수 없습니다.")

CUDA 사용여부 : True
사용 가능한 GPU 수: 1
현재 사용 중인 GPU: NVIDIA GeForce RTX 3090 Ti


In [3]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "beomi/Qwen2.5-7B-Instruct-kowiki-qa",
    max_seq_length = 1024,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.7.1+cu126 with CUDA 1208 (you have 2.7.1+cu118)
    Python  3.9.23 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.3: Fast Qwen2 patching. Transformers: 4.53.2.
   \\   /|    NVIDIA GeForce RTX 3090 Ti. Num GPUs = 1. Max memory: 23.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

In [4]:
# Custom CUDA Settings 

from unsloth import FastModel

model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!
    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [5]:
# 훈련용 chat template 

from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

In [9]:
import json 

with open("instructions_smart_farm.json", encoding="utf-8") as f:
    data = json.load(f)

data

[{'QUESTION': '이인규 교수님은 농업 분야에 언제부터 뛰어들었나요?',
  'ANSWER': '이인규 교수님은 1999년부터 농업 분야에 뛰어들어 25년간 경험을 쌓아오셨습니다.',
  'source': '스마트팜에 입문한 사람은 꼭 시청해야 할 영상｜스마트팜 설명서 EP. 01'},
 {'QUESTION': '스마트팜의 기원은 어디서부터 시작되었나요?',
  'ANSWER': '스마트팜의 기원은 시설 농업, 특히 네덜란드의 유리 온실 농업에서 시작되었습니다.',
  'source': '스마트팜에 입문한 사람은 꼭 시청해야 할 영상｜스마트팜 설명서 EP. 01'},
 {'QUESTION': '네덜란드가 스마트팜을 시작하게 된 환경적 배경은 무엇인가요?',
  'ANSWER': '네덜란드는 간척지로 인해 토양이 연분이 높고, 위도상 추운 기후와 짧은 낮, 긴 밤으로 인해 자연 재배가 어려워서 시설 농업을 발전시켰습니다.',
  'source': '스마트팜에 입문한 사람은 꼭 시청해야 할 영상｜스마트팜 설명서 EP. 01'},
 {'QUESTION': '네덜란드의 초기 유리 온실 농법은 어떤 방식으로 시작되었나요?',
  'ANSWER': '네덜란드의 초기 유리 온실 농법은 보일러를 이용한 난방과 수경 재배를 기반으로 시작되었습니다.',
  'source': '스마트팜에 입문한 사람은 꼭 시청해야 할 영상｜스마트팜 설명서 EP. 01'},
 {'QUESTION': '네덜란드의 수경 재배는 어떤 인공 토양을 사용했나요?',
  'ANSWER': '네덜란드의 수경 재배는 인공 토양인 안면을 사용하였으며, 이는 인공 광물질로 이루어진 깨끗한 배지입니다.',
  'source': '스마트팜에 입문한 사람은 꼭 시청해야 할 영상｜스마트팜 설명서 EP. 01'},
 {'QUESTION': '안면이란 무엇이며, 왜 네덜란드에서 개발되었나요?',
  'ANSWER': '안면은 인공 광물질 배지로, 네덜란드에서 풀잎이나 잡초가 자라는 문제를 해결하기 위해 개발되었습니다

In [15]:
import pandas as pd 


df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

In [19]:
from unsloth.chat_templates import standardize_data_formats

datasets = standardize_data_formats(dataset)

In [22]:
datasets["QUESTION"]

['이인규 교수님은 농업 분야에 언제부터 뛰어들었나요?',
 '스마트팜의 기원은 어디서부터 시작되었나요?',
 '네덜란드가 스마트팜을 시작하게 된 환경적 배경은 무엇인가요?',
 '네덜란드의 초기 유리 온실 농법은 어떤 방식으로 시작되었나요?',
 '네덜란드의 수경 재배는 어떤 인공 토양을 사용했나요?',
 '안면이란 무엇이며, 왜 네덜란드에서 개발되었나요?',
 '네덜란드가 수경 재배 기술을 발전시키기 위해 어떤 연구를 하였나요?',
 '우리나라가 처음 스마트팜 기술을 도입한 시기는 언제인가요?',
 '국내에서 두 번째 스마트팜은 어디에 건립되었나요?',
 '우리나라와 네덜란드의 스마트팜 기술 격차는 어느 정도인가요?',
 '우리나라가 글로벌 스마트팜 경쟁력 순위에서 어느 위치에 있나요?',
 '스마트팜의 기본 에너지원은 무엇인가요?',
 '스마트팜이 온실가스 배출의 주범이 되는 이유는 무엇인가요?',
 '우리나라의 농업 분야 온실가스 배출량은 어느 정도인가요?',
 '지속 가능한 스마트팜을 위해 해결해야 할 과제는 무엇인가요?',
 '미래 농업의 가장 큰 도전 과제는 무엇인가요?',
 '2030년까지 예상되는 세계 인구는 얼마인가요?',
 '미래의 농업에서 농업 인구는 어떻게 변화할 것으로 보시나요?',
 '농업 인구가 감소할 경우 어떤 기술적 대안이 필요하다고 보시나요?',
 '스마트팜의 미래 발전 방향은 무엇이어야 한다고 생각하시나요?',
 '현재 스마트팜이 직면한 가장 큰 문제는 무엇인가요?',
 '미래 농업에서 스마트팜이 반드시 필요한 이유는 무엇인가요?',
 '스마트팜 기술 개발을 위해 우리가 집중해야 할 것은 무엇인가요?',
 '네덜란드와 비교했을 때 우리나라 스마트팜 기술의 차이점은 무엇인가요?',
 '스마트팜이 실현되기 위해 중요한 요소는 무엇인가요?',
 '향후 스마트팜이 지속 가능하려면 어떤 에너지 정책이 필요하다고 보시나요?',
 '기술적 측면에서 스마트팜은 어떤 방향으로 발전해야 하나요?',
 '스마트팜이 인류 식량 문제 해결에 어떤 

In [25]:
def transform_to_qwen25(data):
    return {
        "conversations": [
            {
                "from": "system",
                "value": "너는 농업 전문가야! 농업과 관련된 모든 질문에 정확하고 전문적인 답변을 제공해줘."
            },
            {
                "from": "human", 
                "value": data['QUESTION']
            },
            {
                "from": "gpt",
                "value": data['ANSWER']
            }
        ]
    }

# 2. 데이터셋 변환
transformed_dataset = dataset.map(transform_to_qwen25, remove_columns=['QUESTION', 'ANSWER'])



def formatting_prompts_func(examples):
    """
    Qwen2.5 모델용 프롬프트 포맷팅 함수
    """
    conversations = examples["conversations"]
    texts = []
    
    for conversation in conversations:
        # 각 대화를 Qwen2.5 ChatML 형식으로 변환
        formatted_text = ""
        
        for message in conversation:
            role = message["from"]
            content = message["value"]
            
            if role == "system":
                formatted_text += f"<|im_start|>system\n{content}<|im_end|>\n"
            elif role == "human":
                formatted_text += f"<|im_start|>user\n{content}<|im_end|>\n"
            elif role == "gpt":
                formatted_text += f"<|im_start|>assistant\n{content}<|im_end|>\n"
        
        texts.append(formatted_text)
    
    return {"text": texts}

# 사용 예시
formatted_dataset = transformed_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/7733 [00:00<?, ? examples/s]

Map:   0%|          | 0/7733 [00:00<?, ? examples/s]

In [27]:
formatted_dataset["text"]

['<|im_start|>system\n너는 농업 전문가야! 농업과 관련된 모든 질문에 정확하고 전문적인 답변을 제공해줘.<|im_end|>\n<|im_start|>user\n이인규 교수님은 농업 분야에 언제부터 뛰어들었나요?<|im_end|>\n<|im_start|>assistant\n이인규 교수님은 1999년부터 농업 분야에 뛰어들어 25년간 경험을 쌓아오셨습니다.<|im_end|>\n',
 '<|im_start|>system\n너는 농업 전문가야! 농업과 관련된 모든 질문에 정확하고 전문적인 답변을 제공해줘.<|im_end|>\n<|im_start|>user\n스마트팜의 기원은 어디서부터 시작되었나요?<|im_end|>\n<|im_start|>assistant\n스마트팜의 기원은 시설 농업, 특히 네덜란드의 유리 온실 농업에서 시작되었습니다.<|im_end|>\n',
 '<|im_start|>system\n너는 농업 전문가야! 농업과 관련된 모든 질문에 정확하고 전문적인 답변을 제공해줘.<|im_end|>\n<|im_start|>user\n네덜란드가 스마트팜을 시작하게 된 환경적 배경은 무엇인가요?<|im_end|>\n<|im_start|>assistant\n네덜란드는 간척지로 인해 토양이 연분이 높고, 위도상 추운 기후와 짧은 낮, 긴 밤으로 인해 자연 재배가 어려워서 시설 농업을 발전시켰습니다.<|im_end|>\n',
 '<|im_start|>system\n너는 농업 전문가야! 농업과 관련된 모든 질문에 정확하고 전문적인 답변을 제공해줘.<|im_end|>\n<|im_start|>user\n네덜란드의 초기 유리 온실 농법은 어떤 방식으로 시작되었나요?<|im_end|>\n<|im_start|>assistant\n네덜란드의 초기 유리 온실 농법은 보일러를 이용한 난방과 수경 재배를 기반으로 시작되었습니다.<|im_end|>\n',
 '<|im_start|>system\n너는 농업 전문가야! 농업과 관련된 모든 질문에 정확하고 전문적인 답변을 제공해줘.

In [29]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=24):   0%|          | 0/7733 [00:00<?, ? examples/s]

In [30]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,733 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 20,185,088 of 7,635,801,600 (0.26% trained)


Step,Training Loss
1,2.524600
2,2.578900
3,2.770000
4,2.936100
5,2.504900
6,2.522700
7,2.288900
8,1.853200
9,1.792300
10,1.759700


Unsloth: Will smartly offload gradients to save VRAM!


In [35]:
messages = [
    {
        "role": "system",
        "content": "너는 농업 전문가야! 농업과 관련된 모든 질문에 정확하고 전문적인 답변을 제공해줘."
    },
    {
        "role": "user",
        "content": "토경 재배와 수경 재배의 가장 큰 차이점은 무엇인가요?"
    }
]

# 프롬프트 생성
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False
)

# 토큰화
inputs = tokenizer([text], return_tensors="pt").to("cuda")

# 생성
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

# 디코딩 (입력 부분 제거)
response = tokenizer.decode(outputs[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True)
response.strip()

The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'수경 재배는 토경 재배에 비해 공기와 토양이 필요하지 않으며, 토양이 필요 없는 수경 재배는 토양 병충해로부터 식물을 보호하고, 토양이 필요 없는 재배 방법으로 시설 내 재배가 가능하다는 점이 가장 큰 차이점입니다.'

In [36]:
if True: # Change to True to save finetune!
    model.save_pretrained_merged("Qwen-2.5-ko-instruct", tokenizer)

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Successfully copied all 4 files from cache to Qwen-2.5-ko-instruct.


Unsloth: Merging weights into 16bit: 100%|███████████████████████████████████████████████████| 4/4 [01:50<00:00, 27.73s/it]
